In [173]:
import pandas as pd
import numpy as np

### Load datasets

#### Centers of Population by Census Tract

https://www.census.gov/geographies/reference-files/2010/geo/2010-centers-population.html

In [4]:
df_tx_pop = pd.read_csv("../data/CenPop2010_Mean_TR48")

In [5]:
df_tx_pop.head()

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE
0,48,1,950100,4685,31.999364,-95.531821
1,48,1,950401,5422,31.755614,-95.823901
2,48,1,950402,7535,31.784637,-95.902841
3,48,1,950500,4377,31.776938,-95.631712
4,48,1,950600,6405,31.747594,-95.666055


In [23]:
df_tx_pop.shape

(5265, 6)

### Load lea_tract table

In [155]:
df_lea_tract = pd.read_csv("../data/lea_tract.csv")

In [156]:
df_lea_tract.head()

,LEAID,NAME_LEA19,TRACT
0,100001,Fort Rucker School District (AL),1031010300
1,100001,Fort Rucker School District (AL),1045020000
2,100003,Maxwell AFB School District (AL),1101000900
3,100003,Maxwell AFB School District (AL),1101001000
4,100003,Maxwell AFB School District (AL),1101006000


In [179]:
df_lea_tract.shape

(113520, 6)

In [180]:
df_lea_tract.TRACT.nunique()

74070

### Preprocessing lea_tract

In [158]:
# create tract_only column in df_lea_tract
# take only texas (state code 48)

def select_tract_only(row):
    return int(str(row)[-6:])

def select_state_only(row):
    return int(str(row)[:2])

def select_county(row):
    return int(str(row)[2:5])

df_lea_tract['TRACT_ONLY'] = df_lea_tract['TRACT'].apply(select_tract_only)
df_lea_tract['STATEFP1'] = df_lea_tract['TRACT'].apply(select_state_only)
df_lea_tract['COUNTYFP1'] = df_lea_tract['TRACT'].apply(select_county)

In [159]:
df_lea_tract_tx = df_lea_tract[df_lea_tract.STATEFP1 == 48]
df_lea_tract_tx.shape

(8660, 6)

In [160]:
df_lea_tract_tx.head()

,LEAID,NAME_LEA19,TRACT,TRACT_ONLY,STATEFP1,COUNTYFP1
94939,4800001,Crosbyton Consolidated Independent School Dist...,48107950100,950100,48,107
94940,4800001,Crosbyton Consolidated Independent School Dist...,48107950200,950200,48,107
94941,4800001,Crosbyton Consolidated Independent School Dist...,48169950100,950100,48,169
94942,4800002,Spur Independent School District (TX),48125950300,950300,48,125
94943,4800002,Spur Independent School District (TX),48263950100,950100,48,263


In [161]:
# join tables by df_tx_pop.TRACTCE and df_lea_tract.TRACT_ONLY

df_result_practice = pd.merge(df_lea_tract, df_tx_pop, 
                              how='inner', left_on=['STATEFP1', 'COUNTYFP1', 'TRACT_ONLY'], 
                              right_on=['STATEFP','COUNTYFP','TRACTCE'])

# clean up, drop redundant columns
df_result_practice = df_result_practice.drop(['STATEFP1', 'COUNTYFP1', 'TRACT_ONLY'], axis=1)

In [162]:
df_result_practice.shape

(8660, 9)

In [163]:
df_result_practice.sort_values('LEAID')

,LEAID,NAME_LEA19,TRACT,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE
0,4800001,Crosbyton Consolidated Independent School Dist...,48107950100,48,107,950100,2234,33.648090,-101.227149
2,4800001,Crosbyton Consolidated Independent School Dist...,48107950200,48,107,950200,2346,33.678198,-101.381889
5,4800001,Crosbyton Consolidated Independent School Dist...,48169950100,48,169,950100,6461,33.198856,-101.385674
10,4800002,Spur Independent School District (TX),48263950100,48,263,950100,808,33.247856,-100.629728
8,4800002,Spur Independent School District (TX),48125950300,48,125,950300,2444,33.534982,-100.852578
...,...,...,...,...,...,...,...,...,...
1844,4846770,Zephyr Independent School District (TX),48049950300,48,49,950300,3143,31.768371,-98.837187
8286,4848285,Hallettsville Independent School District (9-1...,48285000400,48,285,400,3586,29.428260,-97.164082
5704,4848285,Hallettsville Independent School District (9-1...,48285000200,48,285,200,3544,29.450155,-96.966021
5212,4899130,Benavides Independent School District (TX),48131950500,48,131,950500,3178,27.527459,-98.397895


In [164]:
df_result_practice.LEAID.nunique()

1022

In [176]:
# sum population by school district
# adding number of census tracts in the district
# NOTE: this is the estimated population for ALL census tracts in the school district, NOT school district population
df_result_pop_tx = df_result_practice.groupby("LEAID").agg({"TRACT": pd.Series.nunique,'POPULATION':np.sum }).reset_index()

In [177]:
# save as csv
df_result_pop_tx.to_csv('../data/tx_pop_by_leaid.txt', index=False)

In [178]:
df_result_pop_tx.head()

,LEAID,TRACT,POPULATION
0,4800001,3,11041
1,4800002,2,3252
2,4800003,2,7607
3,4800005,5,19119
4,4800006,1,1490


### By district, how many unique Service providers serve in the area?

In [189]:
df_d_num_sp = pd.read_csv("../data/tract_num_unique_sp_111920.csv")
df_d_num_sp.head()

,tract,num_unique_sp
0,1001020100,6
1,1001020200,6
2,1001020300,6
3,1001020400,6
4,1001020500,6


In [190]:
df_d_num_sp.shape

(73073, 2)

In [191]:
# merge with leaid, district name, tract population
df_lea_tract.head()

,LEAID,NAME_LEA19,TRACT,TRACT_ONLY,STATEFP1,COUNTYFP1
0,100001,Fort Rucker School District (AL),1031010300,10300,10,310
1,100001,Fort Rucker School District (AL),1045020000,20000,10,450
2,100003,Maxwell AFB School District (AL),1101000900,900,11,10
3,100003,Maxwell AFB School District (AL),1101001000,1000,11,10
4,100003,Maxwell AFB School District (AL),1101006000,6000,11,10


In [192]:
df2 = pd.merge(df_lea_tract[['LEAID', 'NAME_LEA19', 'TRACT']], 
               df_d_num_sp, 
               left_on='TRACT', right_on='tract')

# 
df2.head(20)

,LEAID,NAME_LEA19,TRACT,tract,num_unique_sp
0,100001,Fort Rucker School District (AL),1031010300,1031010300,6
1,100810,Coffee County School District (AL),1031010300,1031010300,6
2,101320,Enterprise City School District (AL),1031010300,1031010300,6
3,100001,Fort Rucker School District (AL),1045020000,1045020000,6
4,102640,Ozark City School District (AL),1045020000,1045020000,6
5,100003,Maxwell AFB School District (AL),1101000900,1101000900,6
6,102430,Montgomery County School District (AL),1101000900,1101000900,6
7,100003,Maxwell AFB School District (AL),1101001000,1101001000,6
8,102430,Montgomery County School District (AL),1101001000,1101001000,6
9,100003,Maxwell AFB School District (AL),1101006000,1101006000,6


In [193]:
df2.shape

(112401, 5)

In [197]:
df2.groupby('LEAID')['num_unique_sp'].sum()

LEAID
100001      12
100003      18
100005      57
100006     133
100007     179
          ... 
5605820      7
5605830     32
5606090      8
5606240     21
6000030     28
Name: num_unique_sp, Length: 13286, dtype: int64

In [195]:
df2[df2.TRACT == 1001020100]

,LEAID,NAME_LEA19,TRACT,tract,num_unique_sp
412,100240,Autauga County School District (AL),1001020100,1001020100,6


In [196]:
df2[df2.LEAID == 100001]

,LEAID,NAME_LEA19,TRACT,tract,num_unique_sp
0,100001,Fort Rucker School District (AL),1031010300,1031010300,6
3,100001,Fort Rucker School District (AL),1045020000,1045020000,6


In [198]:
df3 = pd.read_csv("../data/sp_tract_with_commitment_111920.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [199]:
df3.head(20)

,tract,sp_frn,name,hocoA,committed
0,1001020100,12369286,"HNS License Sub, LLC","Hughes Network Systems, LLC",False
1,1001020100,1857952,"BellSouth Telecommunications, Inc.",AT&T Inc.,True
2,1001020100,18756155,"VSAT Systems, LLC","VSAT Systems, LLC",False
3,1001020100,25646373,"Charter Communications, Inc.",Charter Communications,True
4,1001020100,3766144,"Knology of Alabama, Inc.","WideOpenWest Finance, LLC",True
5,1001020100,4963088,"ViaSat, Inc.","ViaSat, Inc.",False
6,1001020200,12369286,"HNS License Sub, LLC","Hughes Network Systems, LLC",False
7,1001020200,1857952,"BellSouth Telecommunications, Inc.",AT&T Inc.,True
8,1001020200,18756155,"VSAT Systems, LLC","VSAT Systems, LLC",False
9,1001020200,25646373,"Charter Communications, Inc.",Charter Communications,True


In [200]:
df3.shape

(496283, 5)

In [201]:
df3.columns

Index(['tract', 'sp_frn', 'name', 'hocoA', 'committed'], dtype='object')

In [202]:
df_lea_tract.head()

,LEAID,NAME_LEA19,TRACT,TRACT_ONLY,STATEFP1,COUNTYFP1
0,100001,Fort Rucker School District (AL),1031010300,10300,10,310
1,100001,Fort Rucker School District (AL),1045020000,20000,10,450
2,100003,Maxwell AFB School District (AL),1101000900,900,11,10
3,100003,Maxwell AFB School District (AL),1101001000,1000,11,10
4,100003,Maxwell AFB School District (AL),1101006000,6000,11,10


In [220]:
# merge by adding LEAID 

df_sp_by_tract_temp = pd.merge(df3[['tract', 'name', 'hocoA', 'committed']], df_lea_tract[['LEAID', 'NAME_LEA19', 'TRACT']],
              how='inner', left_on = 'tract', right_on = 'TRACT')

# drop redundant TRACT column
df_sp_by_tract_complete.drop('TRACT', axis=1, inplace=True)

In [221]:
df_sp_by_tract_complete.shape

(820275, 6)

In [222]:
df_sp_by_tract_complete.head(10)

,tract,name,hocoA,committed,LEAID,NAME_LEA19
0,1001020100,"HNS License Sub, LLC","Hughes Network Systems, LLC",False,100240,Autauga County School District (AL)
1,1001020100,"BellSouth Telecommunications, Inc.",AT&T Inc.,True,100240,Autauga County School District (AL)
2,1001020100,"VSAT Systems, LLC","VSAT Systems, LLC",False,100240,Autauga County School District (AL)
3,1001020100,"Charter Communications, Inc.",Charter Communications,True,100240,Autauga County School District (AL)
4,1001020100,"Knology of Alabama, Inc.","WideOpenWest Finance, LLC",True,100240,Autauga County School District (AL)
5,1001020100,"ViaSat, Inc.","ViaSat, Inc.",False,100240,Autauga County School District (AL)
6,1001020200,"HNS License Sub, LLC","Hughes Network Systems, LLC",False,100240,Autauga County School District (AL)
7,1001020200,"BellSouth Telecommunications, Inc.",AT&T Inc.,True,100240,Autauga County School District (AL)
8,1001020200,"VSAT Systems, LLC","VSAT Systems, LLC",False,100240,Autauga County School District (AL)
9,1001020200,"Charter Communications, Inc.",Charter Communications,True,100240,Autauga County School District (AL)


In [225]:
df_lea_tract.head()

,LEAID,NAME_LEA19,TRACT,TRACT_ONLY,STATEFP1,COUNTYFP1
0,100001,Fort Rucker School District (AL),1031010300,10300,10,310
1,100001,Fort Rucker School District (AL),1045020000,20000,10,450
2,100003,Maxwell AFB School District (AL),1101000900,900,11,10
3,100003,Maxwell AFB School District (AL),1101001000,1000,11,10
4,100003,Maxwell AFB School District (AL),1101006000,6000,11,10


In [ ]:
# adding pop_tract - population by tract
df_sp_by_tract_complete = 

In [224]:
# save as csv
df_sp_by_tract_complete.to_csv("../data/sp_by_tract_complete.csv", index=False)